In [1]:
from langchain_upstage import ChatUpstage
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from tool import news_tool
from dotenv import load_dotenv
from pprint import pprint
import os

from langchain.globals import set_debug

# set_debug(True)

load_dotenv()
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")

chat = ChatUpstage(model="solar-pro2")
response = chat.invoke("Hello, how are you?")

### Chain으로 최신 뉴스 가져오고 -> 유저 요청에 맞게 응답!

In [2]:
from prompts import SYSTEM_INSTRUCTION

prompt = ChatPromptTemplate.from_messages([
    ("system", "{system}\n\nLatest News Context:\n{news}"),
    HumanMessage(content="{user_input}")
])

def call_news(inputs):
    news = news_tool.invoke({"limit": inputs.get("limit", 5)})
    return {**inputs, "news": news}


chain = (
    RunnablePassthrough()                 # {'user_input', 'limit'}
    | RunnableLambda(call_news)           # {'user_input', 'limit', 'news'}
    | RunnableLambda(                     # 프롬프트에 채우기
          lambda d: {
              "system": SYSTEM_INSTRUCTION,
              "news": d["news"],
              "user_input": d["user_input"],
          })
    | prompt
    | ChatUpstage(model="solar-pro2")     # 실제 LLM 호출
)

result = chain.invoke({"user_input": input()})
print(result.content)

Hello! I can provide you with the latest news summaries based on the context I have. Here's a brief overview of the top stories:

1. **UK-France Migrant Deal**  
   A new "one-in, one-out" pilot scheme has begun, detaining illegal migrants arriving by small boats in the English Channel and returning them to France in exchange for asylum seekers. Critics argue it won't deter crossings, while the government claims it will disrupt people-smuggling gangs.  
   🔗 [Full Article](https://www.bbc.com/news/articles/cewykzegy4qo)

2. **Obesity Jabs & Weight Management**  
   NICE (UK health body) advises post-treatment monitoring for patients using drugs like Wegovy or Mounjaro to prevent weight regain. Trials show patients often regain weight after stopping medication, emphasizing the need for long-term lifestyle support.  
   🔗 [Full Article](https://www.bbc.com/news/articles/cwy3jg20j1ro)

3. **Storm Floris Aftermath**  
   Thousands in Scotland remain without power after the storm caused tra

### LLM이 직접 Tool을 사용하여 뉴스를 가져오기

In [3]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

llm = ChatUpstage(
    model="solar-pro2",
).bind_tools([news_tool])

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "# Overview\n"
     "You are a helpful news bot.\n"
     "Always use the provided tool to fetch fresh news, choosing an appropriate limit.\n"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

agent_runnable = create_tool_calling_agent(
    llm           = llm,
    tools         = [news_tool],
    prompt        = prompt,
)

agent = AgentExecutor(
    
    agent  = agent_runnable,
    tools  = [news_tool],
    verbose=True,
)

result = agent.invoke({"input": input()})

pprint(result)



> Entering new AgentExecutor chain...

Invoking: `NewsTool` with `{'limit': 5}`
responded: 
[This function call is ESSENTIAL to fulfill the "helpful news bot" role by providing the most recent news titles, links, and content as requested. The default limit=5 is appropriate to balance relevance and information quantity without overloading.]

📰 **First migrants could be detained within days under UK-France small boats deal**
🔗 https://www.bbc.com/news/articles/cewykzegy4qo?at_medium=RSS&at_campaign=rss
📄 The UK-French "one-in, one-out" pilot scheme, aimed at reducing the number of small boats crossing the Channel, has come into force with detentions of illegal migrants due to start within days.The deal will see some of those arriving in the UK in small boats detained and returned to France.In exchange the UK will accept from France an equal number of asylum seekers provided they have not already tried to make the crossing and can pass security and eligibility checks.Prime Minister Sir 

### 각 뉴스에 대한 Summary 후 Keyword, Link 추출을 해봅시다!

In [7]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import Field, BaseModel, RootModel
from typing import List

class ArticleSummary(BaseModel):
    summary: str = Field(..., max_length=200)
    keywords: List[str]
    link: str

class Summaries(RootModel[List[ArticleSummary]]):
    pass

json_parser = PydanticOutputParser(pydantic_object=Summaries)


summarize_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are an expert news summarizer.\n"
     "{format_instructions}\n"
     "- Write **one sentence ≤ 25 words** per article (≈ 180 chars max).\n"
     "- Provide 3-5 keywords.\n"
     "- Provide Link\n"),
    ("user", "{articles}")
]).partial(format_instructions=json_parser.get_format_instructions())


summarizer_llm = ChatUpstage(model="solar-pro2", temperature=0)

summarize_chain = summarize_prompt | summarizer_llm | json_parser
pick_articles    = RunnableLambda(lambda r: {"articles": r["output"]})

print("====== SUMMARIZE CHAIN ======")
pprint(summarize_chain)

full_chain = agent | pick_articles | summarize_chain

print("======== FULL CHAIN ========")
pprint(full_chain)

====== SUMMARIZE CHAIN ======
ChatPromptTemplate(input_variables=['articles'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"ArticleSummary": {"properties": {"summary": {"maxLength": 200, "title": "Summary", "type": "string"}, "keywords": {"items": {"type": "string"}, "title": "Keywords", "type": "array"}, "link": {"title": "Link", "type": "string"}}, "required": ["summary", "keywords", "link"], "title": "ArticleSummary", "type": "object"}}, "items": {"$ref": "#/$defs/ArticleSummary"}}\n```'}, messages=[SystemMe

In [5]:
input_text = "최신 7개 뉴스 요약"
result = full_chain.invoke({"input": input_text})



> Entering new AgentExecutor chain...

Invoking: `NewsTool` with `{'limit': 7}`
responded: 
[The NewsTool function is ESSENTIAL because it directly fetches the latest 7 news items including their titles, links, and content as explicitly requested. Without this function call, the question cannot be answered since no general knowledge about "the latest 7 news" exists outside real-time data retrieval.]

📰 **First migrants could be detained within days under UK-France small boats deal**
🔗 https://www.bbc.com/news/articles/cewykzegy4qo?at_medium=RSS&at_campaign=rss
📄 The UK-French "one-in, one-out" pilot scheme, aimed at reducing the number of small boats crossing the Channel, has come into force with detentions of illegal migrants due to start within days.The deal will see some of those arriving in the UK in small boats detained and returned to France.In exchange the UK will accept from France an equal number of asylum seekers provided they have not already tried to make the crossing and

In [6]:
for res in result.root:
    print(f"Summary: {res.summary}")
    print(f"Keywords: {res.keywords}")
    print(f"Links: {res.link}\n")

Summary: The UK and France launch a 'one-in, one-out' migrant scheme to curb Channel crossings, but critics doubt its effectiveness in preventing dangerous journeys.
Keywords: ['UK-France migrant deal', 'small boat crossings', 'asylum seekers']
Links: https://www.bbc.com/news/articles/cewykzegy4qo

Summary: NICE recommends post-treatment check-ups for obesity drug users to prevent weight regain, emphasizing the need for long-term support.
Keywords: ['obesity drugs', 'weight regain', 'NICE guidelines']
Links: https://www.bbc.com/news/articles/cwy3jg20j1ro

Summary: Storm Floris leaves thousands without power in Scotland, with ongoing recovery efforts and travel disruptions.
Keywords: ['Storm Floris', 'power cuts', 'emergency shelters']
Links: https://www.bbc.com/news/articles/c0j9g25q5eyo

Summary: Korean Hiroshima survivors and descendants still face stigma, health issues, and unrecognized suffering decades after the atomic bombing.
Keywords: ['Hiroshima survivors', 'forced labor', 'ra

#### 문제가 있어요...(가끔씩)

Tool 사용을 하면 Link가 그대로 언어모델에게 전달되는데, 한 번 거쳐서 전달되다 보니 Link가 "example.com"으로 떠요.

물론~ 처음부터 tool 사용 가능한 언어모델을 하나만 두고 Output도 구조화된 응답으로 전달하게 하는 chain을 구성하면 됩니다!

아니면 Tool을 직접 사용하지 않게 하고 오로지 패러미터 바인딩만 시키고, Link만 따로 빼낸 후 저장해도 되고요.

간단한 예제여서 이런 상황을 해결하기는 매우 쉽지만, 조금 더 복잡한 경우를 다루어야 할 때에는 **LangGraph**를 사용하는 것이 좋습니다!
